# NOTEBOOK THESIS MARTEN VANDEN BERGHE

### Multispectral monitoring and 3D modeling of the impact of nitrogen deficiency on beans

In [ ]:
folderDATA = '...' # Change ... by user input (change \ to /)

# FUNCTIONS FOR ANALYSIS

## - DATA CONVERSION
## - CAMERA CALIBRATION
## - RED DOT DETECTION
## - 3D COORDINATES & MODELS
## - CORRECTION ON SPECTRAL INDEX
## - COMPARISON WITH AGISOFT SOFTWARE

### LIBRARIES

In [ ]:
# LIBRARIES READ IN 

import matplotlib.pyplot as plt         # Used for creating static, animated, and interactive visualizations in Python
from mpl_toolkits.mplot3d import Axes3D # Enables 3D plotting in Matplotlib
import numpy as np                      # Fundamental package for numerical computations in Python
import cv2                              # Library for computer vision tasks such as image processing, video capture, and analysis
import math                             # Provides access to mathematical functions
import skimage.morphology               # Part of the scikit-image library, contains morphological operations like dilation, erosion, opening, and closing
from skimage import exposure            # Provides functions for adjusting image exposure and contrast
from skimage import io, color           # Used for reading and writing images
from skimage import img_as_ubyte        # Converts images to unsigned byte format
import scipy.ndimage as snd             # Contains functions for multi-dimensional image processing
import scipy.linalg                     # Contains linear algebra functions
import pprint                           # Provides a capability to “pretty-print” arbitrary Python data structures in a well-formatted and readable way
import glob                             # Used for file pattern matching, which can be helpful for reading multiple files
try:                                    # Try to import the png module, used for PNG image processing; pass if it's not installed
    import png
except ImportError:
    pass
import os                               # Provides functions for interacting with the operating system

np.seterr(all='ignore')                 # Set NumPy to ignore all floating-point errors

### FUNCTIONS datImage (by Jan Verwaeren)

In [ ]:
# FUNCTIONS datImage
def read_RGB(IMAGEPATH, dims = (2448,2448,3)):
    """
        Function for reading RGB-bands from CLR.dat file
        
        PARAMETERS
        ----------
        
        IMAGEPATH : str
            absolute or relative path to the .dat file
        
        dims : tuple of ints (m, n, k)
            dimensions of the image 
    """
    f = open(IMAGEPATH, "r")
    a = np.fromfile(f, dtype=np.uint16)
    f.close()
    full = np.reshape(a, dims, order='F')
    return full

def read_SingleBand(IMAGEPATH, dims = (2448,2448), bandNr = 0):
    """
        Function for reading a specific band from a .dat file. Numbers go 
        from 0 to the number of bands in the file minus one (performance issue:
        this function reads the full image and subsequently slices the resulting
        array... performance gains might be obtained when reading selectively ...
        not a real issue at the time of implementation)
        
        PARAMETERS
        ----------
        
        IMAGEPATH : str
            absolute or relative path to the .dat file
        
        dims : tuple of ints (m, n)
            dimensions of the image 
            
        bandNr : int
            band to be read
        
    """
    f = open(IMAGEPATH, "r")
    a = np.fromfile(f, dtype=np.uint16)
    f.close()
    firstNr = (dims[0] * dims[1]) * bandNr
    length = (dims[0] * dims[1])
    full = np.reshape(a[firstNr : firstNr+length], dims, order='F')
    return full
    
def rescale(img, percentiles = None, to_uint8 = False):
    """
        Function that applies histogram stretching on an image, using the
        percentiles of the image histogram percentiles = None. --> to obtain a
        nice-looking image, rescaling is often important!
        
        If percentiles is an (a, b) tuple where a and b are floats, these values
        are used as upper and lower bounds for the stretching.
    """
    if percentiles is None:
            p2, p98 = np.percentile(img, (2, 98))
            percentiles = (p2, p98)
            
    img = exposure.rescale_intensity(img, in_range=percentiles)
    
    if img.dtype == "float64" and img.min() < 0.0:
        img = (img + 1)/2
        
    if to_uint8:
        img = img_as_ubyte(img, force_copy= True)
    return img
        
def save_as_PNG(img, fname , rescale_img = True, percentiles = None):
    """
        Function to save an image array as a png
        
        PARAMETERS
        ----------
        img : np.array
            (m, n, 3) numpy array of RGB values
    """
    if rescale_img:
        img = rescale(img, percentiles = percentiles, to_uint8 = True)
    try:
        io.imsave(fname, img)
    except Exception:
        png.from_array(img, 'RGB').save(fname)

def show_RGB(img, rescale_img = True, percentiles = None):
    if rescale_img:
        img = rescale(img, percentiles = percentiles)
    plt.imshow(np.array(img, dtype=float)/65536)

def gray_to_colormapped(img, cmap = "jet"):
    """
        Function to transform a grayscale image into an RGB image using a particular colormap
        typical applications are heat-map like images (using jet - colormap)
    """
    assert img.dtype == 'uint8', "datatype differs from uint8 when performing gray_to_colormapped"
    if cmap == "jet":
        tmp = np.zeros(list(img.shape) + [3], dtype = 'uint8')
        tmp[:,:,0] = 175-np.array(img/1.46, dtype='uint8')
        tmp[:,:,1] = 255
        tmp[:,:,2] = 255
        return img_as_ubyte(color.hsv2rgb(tmp))
    
    elif cmap == "jet inv":
        tmp = np.zeros(list(img.shape) + [3], dtype = 'uint8')
        tmp[:,:,0] = np.array(img/1.46, dtype='uint8')
        tmp[:,:,1] = 255
        tmp[:,:,2] = 255
        return img_as_ubyte(color.hsv2rgb(tmp))
    
    elif cmap == "fv/fm":
        tmp = np.zeros(list(img.shape) + [3], dtype = 'uint8')
        tmp[:,:,0] = np.array(img/3.5, dtype='uint8')
        tmp[:,:,1] = 255
        tmp[:,:,2] = 255
        return img_as_ubyte(color.hsv2rgb(tmp))
    
    elif cmap =='gray':
        return img_as_ubyte(color.gray2rgb(img), force_copy=True)

def convert_folder_RGB(img_path_list, percentiles = None):
    cnt = 0
    for fname in img_path_list:
        try:
            img = read_RGB(fname)
            save_as_PNG(img, fname[:-3] + "png", rescale_img = True, percentiles = percentiles)
            cnt += 1
        except Exception:
            pass
    return cnt

def computeImage(image_dict, expression_str):
    """
    Computes the result of computing the expression in expression_str. The keys in image_dict are the variables names in the expression and the values the associated images
    
    TAKE CARE: uses some dirty tricks to avoid the need for tailor-made parser of expressions
    
    """
    d = image_dict
    for key in image_dict:
        expression_str = expression_str.replace(key, "d['" + key + "']")
    
    res = eval(expression_str)
    return res

def rescale_after_computation(im , rescale_method = "bound_to_unitInterval"):
    if rescale_method == "bound_to_unitInterval":
        pass
    elif rescale_method == "use_max":
        return im / np.max(im)
    else:
        return im

def parseExpression(expression, img_path):
    """
        Function to parse an expression and execute it at 'run time'. 
        
        Typical function for the DATimage_GUI --> should be relocated in future implementations
        
        RETURNS
        -------
        
        an (m, n) numpy array
        
    """
    fileTypes_dict = dict()
    # get list of bands
    tmp_expression = expression.replace(']', '] ')
    tmp_expression = ''.join([c for c in tmp_expression if not c in ['+', '-', '*', '/', '(', ')']])
    bands_lst = tmp_expression.split()
    
    # read bands from dat-files
    for band in bands_lst:
        try:
            referenceName = extractReferenceName(img_path)
            dat_file =  os.path.dirname(img_path)+ '/' + band[:3] + referenceName
            nr  = int(band[4])
            single_band = read_SingleBand(dat_file, bandNr = nr)
            fileTypes_dict[band.strip()] = np.array(single_band, dtype = float)
        except Exception as e:
            print("problem during parsing of band (may be ignored if the printed value is not a band)", band, e.__str__())
        
    computed = computeImage(fileTypes_dict, expression)
    computed[np.isnan(computed)] = 0
    computed[np.isinf(computed)] = 0
    return computed

def extractReferenceName(img_path):
    baseName = os.path.basename(img_path)
    return baseName[3:]

### LOADING DATA

In [ ]:
# FUNCTION - change/create pathname
def make_path_recursively(path):
    """
    Creates directories recursively if they don't already exist.

    Parameters:
    path (str): The path of directories to be created.
    """
    folders = path.split('/')
    for f in range(2,len(folders)+1):
        folder = '/'.join(folders[:f])
        if not os.path.exists(folder):
            print('Making', folder)
            os.mkdir(folder)

In [ ]:
# Data folders

# DATA CONVERSION
CLR_files_path = f'{folderDATA}/DATA_THESIS_MARTEN/MSI/20240314_Spectral_RGB_CLR/*.DAT'
imagefolderOut = f'{folderDATA}/DATA_THESIS_MARTEN/MSI/Edited_images_17DAS'
if not os.path.exists(imagefolderOut):
    os.mkdir(imagefolderOut)

# The used rows and columns of the MSI
row = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']
col = ['1', '2', '3', '4', '5', '6', '7', '8', '9']

# The used labels for the replicates and the treatments
replicates = ['1', '2', '3', '4', '5', '6', '7', '8']
treatments = ['0%', '10%', '25%', '50%', '75%', '100%']

# List creation of the positions of the images
left = list(range(12))
mid = list(range(12, 96))
right = list(range(96, 108)) 

# CAMERA CALIBRATION
calibfolderIn = f'{folderDATA}/DATA_THESIS_MARTEN/MSI/Camera_calibration/Data'
calibfolderOut = f'{folderDATA}/DATA_THESIS_MARTEN/MSI/Camera_calibration/Matrix'
make_path_recursively(calibfolderOut)
CHECKERBOARD = (6,8)
visualize = True

# IMAGES INDIVIDUAL 25%N-3
# !!! LOAD-ORDER VERY IMPORTANT !!!
# The images were first visually double-checked to make sure no (parts of) other plants were visible 
# Order image_paths/titles = [TopLeft, TopRight, BottomLeft, BottomRight]
image_paths = [f'{imagefolderOut}/E4_25%_3.jpg',
               f'{imagefolderOut}/F4_25%_3.jpg',
               f'{imagefolderOut}/E3_25%_3.jpg',
               f'{imagefolderOut}/F3_25%_3.jpg',]
titles = ["E4", "F4", "E3", "F3"]



### CONVERSION CLR.DAT-files TO JPG-files

In [ ]:
# FUNCTION - clr to jpg conversion
def CLR_to_JPG(CLR_files_path, imagefolderOut, row, col, replicates, treatments, left, mid, right, visualize=False):
    """
    Convert CLR.DAT files to JPG images, applying specific transformations based on their position (left, mid, right).

    Parameters:
    CLR_files_path (str): Path pattern to the CLR.DAT files.
    imagefolderOut (str): Output folder where images are saved.
    row (list): List of number/name of rows.
    col (list): List of number/name of columns.
    replicates (list): List of number/name of replicates.
    treatments (list): List of number/name of treatments.
    left (list): List of indices of images on the left side.
    mid (list): List of indices of images in the middle.
    right (list): List of indices of images on the right side.
    visualize (bool): Whether to visualize the files. Defaults to False.
    """
    CLR_files_path = sorted(glob.glob(CLR_files_path))
    if not os.path.exists(imagefolderOut):
        os.makedirs(imagefolderOut)
    
    sides = left + right

    for i in range(len(CLR_files_path)):
        if i in mid:
            row_index = i % len(row)
            col_index = (i // 12)
            tre_index = (i // 2) % len(treatments)
            rep_index = (i // 12) - 1
            title = f"{row[row_index]}{col[col_index]}_{treatments[tre_index]}_{replicates[rep_index]}"
            image = rescale(read_RGB(CLR_files_path[i]), percentiles=None, to_uint8=True)
            image[0:image.shape[0] // 2 - 100, :, :] = 0
            if visualize:
                plt.figure(figsize=(10, 5))
                plt.imshow(image)
                plt.title(title)
                plt.show()
            plt.imsave(os.path.join(imagefolderOut, f"{title}.jpg"), image)

            row_index = i % len(row)
            col_index = (i // 12)
            tre_index = (i // 2) % len(treatments)
            rep_index = (i // 12)
            title = f"{row[row_index]}{col[col_index]}_{treatments[tre_index]}_{replicates[rep_index]}"
            image = rescale(read_RGB(CLR_files_path[i]), percentiles=None, to_uint8=True)
            image[image.shape[0] // 2:, :, :] = 0
            if visualize:            
                plt.figure(figsize=(10, 5))
                plt.imshow(image)
                plt.title(title)
                plt.show()
            plt.imsave(os.path.join(imagefolderOut, f"{title}.jpg"), image)

        if i in sides:
            if i in left:
                row_index = i % len(row)
                col_index = (i // 12)
                tre_index = (i // 2) % len(treatments)
                rep_index = (i // 12)
                title = f"{row[row_index]}{col[col_index]}_{treatments[tre_index]}_{replicates[rep_index]}"
                image = rescale(read_RGB(CLR_files_path[i]), percentiles=None, to_uint8=True)
                if visualize:                
                    plt.figure(figsize=(10, 5))
                    plt.imshow(image)
                    plt.title(title)
                    plt.show()
                plt.imsave(os.path.join(imagefolderOut, f"{title}.jpg"), image)

            if i in right:
                row_index = i % len(row)
                col_index = (i // 12)
                tre_index = (i // 2) % len(treatments)
                rep_index = (i // 12) - 1
                title = f"{row[row_index]}{col[col_index]}_{treatments[tre_index]}_{replicates[rep_index]}"
                image = rescale(read_RGB(CLR_files_path[i]), percentiles=None, to_uint8=True)
                image[2300:image.shape[0], :, :] = 0
                if visualize:
                    plt.figure(figsize=(10, 5))
                    plt.imshow(image)
                    plt.title(title)
                    plt.show()
                plt.imsave(os.path.join(imagefolderOut, f"{title}.jpg"), image)


In [ ]:
# RUN - clr to jpg conversion 17 DAS
CLR_to_JPG(CLR_files_path, imagefolderOut, row, col, replicates, treatments, left, mid, right, True)

In [ ]:
# RUN - clr to jpg conversion 15 DAS
CLR_files_path_15DAS = f'{folderDATA}/DATA_THESIS_MARTEN/MSI/20240312_Spectral_RGB_CLR/*.DAT'
imagefolderOut_15DAS = f'{folderDATA}/DATA_THESIS_MARTEN/MSI/Edited_images_15DAS'

CLR_to_JPG(CLR_files_path_15DAS, imagefolderOut_15DAS, row, col, replicates, treatments, left, mid, right, True)

### CAMERA CALIBRATION + INFORMATION

In [ ]:
# FUNCTION - camera calibration
def camera_calibration(calibfolderIn, calibfolderOut, CHECKERBOARD=(6,8), visualize=True):
    """
    Function:
    - Camera calibration using OpenCv
    Parameters:
     - calibfolderIn (str): Folder with images of checkerboard (CLR.DAT)
     - calibfolderOut (str): Folder where the calibration matrices are stored
     - CHECKERBOARD (tuple): Dimensions of the checkerboard (rows-1, columns-1)
     - visualize (bool): True to see the detection of the checkerboard on the images
    Returns:
    - mtx (ndarray): Calibration matrix
    - dist (ndarray): Distortion matrix
    """
    
    filesCLR = glob.glob(f'{calibfolderIn}/CLR*DAT')

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Creating vector to store vectors of 3D points for each checkerboard image
    objpoints = []
    # Creating vector to store vectors of 2D points for each checkerboard image
    imgpoints = [] 

    # Defining the world coordinates for 3D points
    objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
    objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
    prev_img_shape = None

    for fname in filesCLR:
        print(f"Processing file: {fname}")
        
        try:
            img = rescale(read_RGB(fname), percentiles=None, to_uint8=True)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        except Exception as e:
            print(f"Error reading or processing image {fname}: {e}")
            continue

        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH +
                                                 cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
        
        if ret:
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)
            
            if visualize:
                plt.figure()
                plt.imshow(gray, cmap='gray')
                plt.plot(corners2[:,0,0], corners2[:,0,1], 'o-')
                plt.title(f"Detected corners for {fname}")
                plt.show()
        else:
            print(f"Chessboard not detected in image {fname}")
            
            if visualize:
                plt.figure()
                plt.imshow(gray, cmap='gray')
                plt.title(f"No corners detected for {fname}")
                plt.show()

        h, w = img.shape[:2]

    # Performing camera calibration by 
    # passing the value of known 3D points (objpoints)
    # and corresponding pixel coordinates of the 
    # detected corners (imgpoints)
    ret, mtx, dist, _, _ = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
    np.save(f'{calibfolderOut}/mtx.npy', mtx, allow_pickle=True, fix_imports=True)
    np.save(f'{calibfolderOut}/dist.npy', dist, allow_pickle=True, fix_imports=True)
    
    return mtx, dist


In [ ]:
# RUN - calibration/distortie matrix 
mtx, dist = camera_calibration(calibfolderIn, calibfolderOut, CHECKERBOARD, visualize)
print("Camera matrix: \n", mtx)
print("Distortion coefficients: \n", dist)


In [ ]:
# CALIBRATION RASPBERRY PI
# FUNCTION - camera calibration
def camera_calibration_raspberry(calibfolderIn, calibfolderOut, CHECKERBOARD=(6,8), visualize=True):
    """
    Function:
    - Camera calibration using OpenCv
    Parameters:
     - calibfolderIn (str): Folder with images of checkerboard (JPG)
     - calibfolderOut (str): Folder where the calibration matrices are stored
     - CHECKERBOARD (tuple): Dimensions of the checkerboard (rows-1, columns-1)
     - visualize (bool): True to see the detection of the checkerboard on the images
    Returns:
    - mtx (ndarray): Calibration matrix
    - dist (ndarray): Distortion matrix
    """
    
    filesJPG = glob.glob(f'{calibfolderIn}/*.JPG')

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Creating vector to store vectors of 3D points for each checkerboard image
    objpoints = []
    # Creating vector to store vectors of 2D points for each checkerboard image
    imgpoints = [] 

    # Defining the world coordinates for 3D points
    objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
    objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
    prev_img_shape = None

    for fname in filesJPG:
        print(f"Processing file: {fname}")
        
        try:
            img = cv2.imread(fname)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        except Exception as e:
            print(f"Error reading or processing image {fname}: {e}")
            continue

        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH +
                                                 cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
        
        if ret:
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)
            
            if visualize:
                plt.figure()
                plt.imshow(gray, cmap='gray')
                plt.plot(corners2[:,0,0], corners2[:,0,1], 'o-')
                plt.title(f"Detected corners for {fname}")
                plt.show()
        else:
            print(f"Chessboard not detected in image {fname}")
            
            if visualize:
                plt.figure()
                plt.imshow(gray, cmap='gray')
                plt.title(f"No corners detected for {fname}")
                plt.show()

        h, w = img.shape[:2]

    # Performing camera calibration by 
    # passing the value of known 3D points (objpoints)
    # and corresponding pixel coordinates of the 
    # detected corners (imgpoints)
    ret, mtx, dist, _, _ = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
    np.save(f'{calibfolderOut}/mtx.npy', mtx, allow_pickle=True, fix_imports=True)
    np.save(f'{calibfolderOut}/dist.npy', dist, allow_pickle=True, fix_imports=True)
    
    return mtx, dist


In [ ]:
# RUN - Calibration Raspberry cams

# HQ cam
calibfolderIn_rasp_HQ = f'{folderDATA}/DATA_THESIS_MARTEN/RaspberryPi/HQ_cam/Data'
calibfolderOut_rasp_HQ = f'{folderDATA}/DATA_THESIS_MARTEN/RaspberryPi/HQ_cam/Matrix'
mtx_rasp_HQ, dist_rasp_HQ = camera_calibration_raspberry(calibfolderIn_rasp_HQ, calibfolderOut_rasp_HQ, CHECKERBOARD, visualize)
print("Camera matrix: \n", mtx_rasp_HQ)
print("Distortion coefficients: \n", dist_rasp_HQ)

# V2 cam
calibfolderIn_rasp_V2 = f'{folderDATA}/DATA_THESIS_MARTEN/RaspberryPi/V2_cam/Data'
calibfolderOut_rasp_V2 = f'{folderDATA}/DATA_THESIS_MARTEN/RaspberryPi/V2_cam/Matrix'
mtx_rasp_V2, dist_rasp_V2 = camera_calibration_raspberry(calibfolderIn_rasp_V2, calibfolderOut_rasp_V2, CHECKERBOARD, visualize)
print("Camera matrix: \n", mtx_rasp_V2)
print("Distortion coefficients: \n", dist_rasp_V2)

In [ ]:
# FUNCTION - Undistortion images

def undistortion(calibfolderIn, calibfolderOut, mtx, dist):
    """
    Function:
    - Undistort images using the provided camera matrix and distortion coefficients
    Parameters:
     - calibfolderIn (str): Folder with images to be undistorted
     - calibfolderOut (str): Folder where the undistorted images will be saved
     - mtx (ndarray): Camera matrix
     - dist (ndarray): Distortion coefficients
    """
    if not os.path.exists(calibfolderOut):
        os.makedirs(calibfolderOut)
        
    # Pattern to match JPG files
    filesJPG = glob.glob(f'{calibfolderIn}/*.JPG')

    for fname in filesJPG:
        print(f"Processing file: {fname}")
        img = cv2.imread(fname)
        h, w = img.shape[:2]
        newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

        # Undistort the image
        dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

        # Crop the image
        x, y, w, h = roi
        dst = dst[y:y+h, x:x+w]

        # Construct the output file path
        base_name = os.path.basename(fname)
        output_path = os.path.join(calibfolderOut, f'undistorted_{base_name}')

        # Save the undistorted image
        cv2.imwrite(output_path, dst)

        print(f"Saved undistorted image to: {output_path}")
        
        # Visualize the original and undistorted images
        plt.figure(figsize=(10, 5))

        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title('Original Image')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(dst, cv2.COLOR_BGR2RGB))
        plt.title('Undistorted Image')
        plt.axis('off')

        plt.show()


In [ ]:
# RUN - Undistortion HQ cam
undistortfolderIn_rasp_HQ = f'{folderDATA}/DATA_THESIS_MARTEN/RaspberryPi/HQ_cam/Data'
undistortfolderOut_rasp_HQ = f'{folderDATA}/DATA_THESIS_MARTEN/RaspberryPi/HQ_cam/Undistort'

mtx_HQ = np.load("D:/THESIS/Camera_Calibration/RaspberryPi/HQ_cam/Matrix/mtx.npy")
dist_HQ = np.load("D:/THESIS/Camera_Calibration/RaspberryPi/HQ_cam/Matrix/dist.npy")

undistortion(undistortfolderIn_rasp_HQ, undistortfolderOut_rasp_HQ, mtx_HQ, dist_HQ)

# RUN - Undistortion V2 cam
undistortfolderIn_rasp_V2 = f'{folderDATA}/DATA_THESIS_MARTEN/RaspberryPi/V2_cam/Data'
undistortfolderOut_rasp_V2 = f'{folderDATA}/DATA_THESIS_MARTEN/RaspberryPi/V2_cam/Undistort'

mtx_V2 = np.load("D:/THESIS/Camera_Calibration/RaspberryPi/V2_cam/Matrix/mtx.npy")
dist_V2 = np.load("D:/THESIS/Camera_Calibration/RaspberryPi/V2_cam/Matrix/dist.npy")

undistortion(undistortfolderIn_rasp_V2, undistortfolderOut_rasp_V2, mtx_V2, dist_V2)


In [ ]:
# RUN - Undistortion MSI
undistortfolderIn_MSI = f'{folderDATA}/DATA_THESIS_MARTEN/MSI/Camera_calibration/Data'
undistortfolderOut_MSI = f'{folderDATA}/DATA_THESIS_MARTEN/MSI/Camera_calibration/Undistort'
mtx_MSI = np.load("D:/THESIS/Camera_Calibration/MSI/Matrix/mtx.npy")
dist_MSI = np.load("D:/THESIS/Camera_Calibration/MSI/Matrix/dist.npy")

undistortion(undistortfolderIn_MSI, undistortfolderOut_MSI, mtx_MSI, dist_MSI)

In [ ]:
# INFORMATION READ IN

# Pixel density
pixel_width = 6.12e-3 #infosheet PathoViewer (15mm/2448pixels)

# Calibration matrix
print("Calibration matrix:")
print(mtx)

# Coordinates principal point
center = [mtx[0,2], mtx[1,2]]
print("Center image (xp,yp):")
xp_pix = center[0] # x_principal_point
yp_pix = center[1] # y_principal_point
print("xp_pix:", xp_pix, "yp_pix:", yp_pix)

xp_mm = xp_pix * pixel_width
yp_mm = yp_pix * pixel_width
print("xp_mm:", xp_mm, "yp_mm:", yp_mm)
print()

# Calculate focal distance as mean in mm
f_pix = (mtx[0,0] + mtx[1,1])/2
f_mm = pixel_width * f_pix
print("focal_dist(pix):", f_pix)
print("focal_dist(mm):",f_mm)
print()

# Pinholes camera
pinhole = np.array([[ -40, 0, 0],[40, 0, 0]])
print("Pinholes:")
print(pinhole)
print()

# Number of cameras
n_cam = pinhole.shape[0]
print("Number of camera angles:",n_cam)

### SHOW IMAGES WITH TITLES

In [ ]:
# FUNCTION - show images
def show_images(image_paths, titles):
    """
    Displays images from the given file paths with their respective titles.

    Parameters:
    - image_paths (list of str): File paths of the images to be displayed.
    - titles (list of str): Titles to be used for each image.
    """
    # Ensure that the number of image paths matches the number of titles
    assert len(image_paths) == len(titles), "Number of image paths must match the number of titles"

    # Read and convert the images from BGR to RGB
    images_rgb = [cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB) for path in image_paths]

    # Plot the images
    plt.figure(figsize=(20, 20))
    for i, (image_rgb, title) in enumerate(zip(images_rgb, titles), start=1):
        plt.subplot(2, 2, i)
        plt.imshow(image_rgb)
        plt.title(title)
    plt.show()


In [ ]:
# RUN - show images
show_images(image_paths, titles)

### RED DOT DETECTION

In [ ]:
# FUNCTION - red dot detection
def find_red_dots(image):
    """
    Detects red dots in an image and returns their coordinates.

    Parameters:
    - image: The RGB image as a numpy array.

    Returns:
    - numpy array of coordinates of detected red dots.
    """
    min_red = np.array([170, 0, 0])  # min. limit Red (RGB)
    max_red = np.array([255, 80, 80])  # max. limit Red (RGB)
    mask = cv2.inRange(image, min_red, max_red)

    image_dilated = snd.morphology.binary_dilation(mask, iterations=20)

    mask_labeled = skimage.morphology.label(image_dilated)

    # Coordinates of centers of objects, filter out too small and too large objects
    min_size = 20
    max_size = 10000
    coords_pred = []
    for o in sorted(np.unique(mask_labeled))[1:]:
        if min_size <= np.sum(mask_labeled == o) <= max_size:
            coords = list(np.mean(np.argwhere(mask_labeled == o), axis=0))
            coords_pred.append(coords)

    return np.array(coords_pred)

# FUNCTION - labelling red dots on images
def find_red_dots_in_images(image_paths, titles):
    """
    Detects red dots in multiple images and prints their coordinates along with titles.

    Parameters:
    - image_paths: List of paths to the images.
    - titles: List of titles for the images.

    Returns:
    - None
    """
    # Read and convert the images from BGR to RGB
    images_rgb = [cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB) for path in image_paths]

    # Detect red dots in each image
    red_dots = [find_red_dots(image_rgb) for image_rgb in images_rgb]

    # Print coordinates of the detected red dots
    for title, dots in zip(titles, red_dots):
        print(f"Coordinates red dots: {title}")
        print(dots)

# FUNCTION - plot red dots
def plot_coordinates_red_dots(image_paths, titles):
    """
    Detects red dots in the given images and plots the results.

    Parameters:
    - image_paths: List of strings, file paths of the images.
    - titles: List of strings, titles to be used for each image.
    """
    # Read and convert the images from BGR to RGB
    images_rgb = [cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB) for path in image_paths]

    # Detect red dots in each image
    red_dots = [find_red_dots(image_rgb) for image_rgb in images_rgb]

    # Plot the images with detected red dots
    plt.figure(figsize=(20, 20))
    for i, (image_rgb, dots, title) in enumerate(zip(images_rgb, red_dots, titles), start=1):
        plt.subplot(2, 2, i)
        plt.imshow(image_rgb)
        X = [c[1] for c in dots]
        Y = [c[0] for c in dots]
        labels = [chr(ord('A') + j) for j in range(len(dots))]
        for x, y, label in zip(X, Y, labels):
            plt.scatter(x, y, color='cyan', marker='x')
            plt.text(x, y, label, color='yellow', fontsize=12, ha='center', va='center')
        plt.title(title)
    plt.show()


In [ ]:
# RUN - red dot detection/plot
find_red_dots_in_images(image_paths, titles)
plot_coordinates_red_dots(image_paths, titles)

### MANUAL MATCHING

In [ ]:
# RUN - manual matching the detected points of the images 

# titles = [TopLeft, TopRight, BottomLeft, BottomRight]
# Dataset left leaf
letter_coordinates_left = {'A1': {f'x_{titles[2]}': 82.47543161, f'y_{titles[2]}': 1978.92895086, f'x_{titles[0]}': 1425.78976982, f'y_{titles[0]}': 1990.79539642},      
                           'B1': {f'x_{titles[2]}': 139.30730184, f'y_{titles[2]}': 1871.91839125, f'x_{titles[0]}': 1457.89690265, f'y_{titles[0]}': 1885.57433628},
                           'C1': {f'x_{titles[2]}': 390.07896679, f'y_{titles[2]}': 2156.77269373, f'x_{titles[0]}': 1740.19178982, f'y_{titles[0]}': 2173.01543514},
                           'D1': {f'x_{titles[2]}': 391.0218503, f'y_{titles[2]}': 2056.9739656, f'x_{titles[0]}': 1734.52584856, f'y_{titles[0]}': 2076.56344648},
                           'E1': {f'x_{titles[2]}': 369.87438768, f'y_{titles[2]}': 1945.85549335, f'x_{titles[0]}': 1703.37221135, f'y_{titles[0]}': 1962.58395303},
                           'F1': {f'x_{titles[2]}': 371.14505609, f'y_{titles[2]}': 1836.71327942, f'x_{titles[0]}': 1688.05529954, f'y_{titles[0]}': 1853.88302021},
                           'G1': {f'x_{titles[2]}': 568.19253112, f'y_{titles[2]}': 1783.44232365, f'x_{titles[0]}': 1868.38286796, f'y_{titles[0]}': 1793.46758164},
                           'H1': {f'x_{titles[2]}': 731.93969368, f'y_{titles[2]}': 2111.82386726, f'x_{titles[0]}': 2074.39584183, f'y_{titles[0]}': 2125.80105993}}
# Dataset right leaf
letter_coordinates_right = {'A2': {f'x_{titles[3]}': 257.12375691, f'y_{titles[3]}': 940.10055249, f'x_{titles[1]}': 1549.46332335, f'y_{titles[1]}': 963.62075848},
                            'B2': {f'x_{titles[3]}': 258.89736842, f'y_{titles[3]}': 479.80842105, f'x_{titles[1]}': 1649.21178637, f'y_{titles[1]}': 504.26120319},
                            'C2': {f'x_{titles[3]}': 330.99398108, f'y_{titles[3]}': 987.72656922, f'x_{titles[1]}': 1610.09143244, f'y_{titles[1]}': 1009.8205215},
                            'D2': {f'x_{titles[3]}': 425.22744479, f'y_{titles[3]}': 1030.46182965, f'x_{titles[1]}': 1702.91860465, f'y_{titles[1]}': 1046.30268895},
                            'E2': {f'x_{titles[3]}': 463.47639797, f'y_{titles[3]}': 603.06100218, f'x_{titles[1]}': 1808.43844732, f'y_{titles[1]}': 625.39926063},
                            'F2': {f'x_{titles[3]}': 488.10477941, f'y_{titles[3]}': 705.89154412, f'x_{titles[1]}': 1822.88, f'y_{titles[1]}': 725.90981818},
                            'G2': {f'x_{titles[3]}': 525.98148148, f'y_{titles[3]}': 821.00420875, f'x_{titles[1]}': 1852.04826118, f'y_{titles[1]}': 836.079489},
                            'H2': {f'x_{titles[3]}': 539.91213969, f'y_{titles[3]}': 895.44068736, f'x_{titles[1]}': 1839.82267833, f'y_{titles[1]}': 909.74461642},
                            'I2': {f'x_{titles[3]}': 568.18833203, f'y_{titles[3]}': 964.16092404, f'x_{titles[1]}': 1837.1892562, f'y_{titles[1]}': 981.7},
                            'J2': {f'x_{titles[3]}': 771.31919275, f'y_{titles[3]}': 832.41062603, f'x_{titles[1]}': 2072.32609819, f'y_{titles[1]}': 847.94418605},
                            'K2': {f'x_{titles[3]}': 812.8172043, f'y_{titles[3]}': 751.70430108, f'x_{titles[1]}': 2130.10507881, f'y_{titles[1]}': 767.38558085},
                            'L2': {f'x_{titles[3]}': 769.80253067, f'y_{titles[3]}': 580.08090491, f'x_{titles[1]}': 2118.48322627, f'y_{titles[1]}': 598.38329764}}


print("Letter Coordinates Left:")
pprint.pprint(letter_coordinates_left)

print("Letter Coordinates Right:")
pprint.pprint(letter_coordinates_right)


In [ ]:
# FUNCTION - plots matched coordinates
def plot_matched_coordinates(image_paths, letter_coordinates_left, letter_coordinates_right, titles):
    """
    Plots matched points on the images with their respective labels.

    Parameters:
    - image_paths: List of strings, file paths of the images.
    - letter_coordinates_left: Dictionary containing coordinates for matched points in the left images.
    - letter_coordinates_right: Dictionary containing coordinates for matched points in the right images.
    - titles: List of strings, titles to be used for each image.
    """
    # Read and convert the images from BGR to RGB
    images_rgb = [cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB) for path in image_paths]

    plt.figure(figsize=(20, 20))

    for i, (image_rgb, title) in enumerate(zip(images_rgb, titles), start=1):
        plt.subplot(2, 2, i)
        plt.imshow(image_rgb)
        
        # Determine which coordinates to use based on the current title
        if title in [titles[0], titles[2]]:
            letter_coordinates = letter_coordinates_left
        else:
            letter_coordinates = letter_coordinates_right
        
        for point, coords in letter_coordinates.items():
            y = coords[f'x_{title}']
            x = coords[f'y_{title}']
            plt.scatter(x, y, color='yellow', marker='x')
            plt.text(x, y, point, color='cyan', fontsize=12, ha='center', va='bottom')
        plt.title(title)

    plt.show()


In [ ]:
# RUN - plot matched coordinates
plot_matched_coordinates(image_paths, letter_coordinates_left, letter_coordinates_right, titles)

### PIXELCOORDS & WORLDCOORDS

In [ ]:
# FUNCTION - define pixel coordinates 
def pixel_coords(titles, letter_coordinates_left, letter_coordinates_right):
    """
    Creates datasets with pixel coordinates for every image.

    Parameters:
    - titles: List of strings, titles to be used for each image.
    - letter_coordinates_left: Dictionary containing coordinates for matched points in the left images.
    - letter_coordinates_right: Dictionary containing coordinates for matched points in the right images.

    Returns:
    - A dictionary with titles as keys and corresponding pixel coordinates as values.
    """
    pix_coords = {title: {} for title in titles}

    for key in letter_coordinates_left:
        pix_coords[titles[0]][key] = (letter_coordinates_left[key][f'x_{titles[0]}'], letter_coordinates_left[key][f'y_{titles[0]}'])
        pix_coords[titles[2]][key] = (letter_coordinates_left[key][f'x_{titles[2]}'], letter_coordinates_left[key][f'y_{titles[2]}'])

    for key in letter_coordinates_right:
        pix_coords[titles[1]][key] = (letter_coordinates_right[key][f'x_{titles[1]}'], letter_coordinates_right[key][f'y_{titles[1]}'])
        pix_coords[titles[3]][key] = (letter_coordinates_right[key][f'x_{titles[3]}'], letter_coordinates_right[key][f'y_{titles[3]}'])

    return pix_coords


In [ ]:
# RUN -  define pixel coordinates for every image
pixel_coords = pixel_coords(titles, letter_coordinates_left, letter_coordinates_right)

for title in titles:
    print(f"pixel coordinates: {title}")
    pprint.pprint(pixel_coords[title])

In [ ]:
# FUNCTION - define world coordinates
def world_coords(titles, pix_coords):
    """
    Converts pixel coordinates to world coordinates.

    Parameters:
    - titles: List of strings, titles to be used for each image.
    - pix_coords: Dictionary containing pixel coordinates for each image.

    Returns:
    - A dictionary with titles as keys and lists of corresponding 3D world coordinates.
    """
    world_coords = {title: [] for title in titles}

    for key in pix_coords[titles[0]].keys():
        x_im1 = (pix_coords[titles[0]][key][0] - xp_pix) * pixel_width + pinhole[1][0]
        y_im1 = (pix_coords[titles[0]][key][1] - yp_pix) * pixel_width + pinhole[1][1]
        z_im1 = f_mm + pinhole[1][1]
        world_coords[titles[0]].append([key, x_im1, y_im1, z_im1])
        
        x_im3 = (pix_coords[titles[2]][key][0] - xp_pix) * pixel_width + pinhole[0][0]
        y_im3 = (pix_coords[titles[2]][key][1] - yp_pix) * pixel_width + pinhole[0][1]
        z_im3 = f_mm + pinhole[0][2]
        world_coords[titles[2]].append([key, x_im3, y_im3, z_im3])
        
    for key in pix_coords[titles[1]].keys():
        x_im2 = (pix_coords[titles[1]][key][0] - xp_pix) * pixel_width + pinhole[1][0]
        y_im2 = (pix_coords[titles[1]][key][1] - yp_pix) * pixel_width + pinhole[1][1]
        z_im2 = f_mm + pinhole[1][2]
        world_coords[titles[1]].append([key, x_im2, y_im2, z_im2])
        
        x_im4 = (pix_coords[titles[3]][key][0] - xp_pix) * pixel_width + pinhole[0][0]
        y_im4 = (pix_coords[titles[3]][key][1] - yp_pix) * pixel_width + pinhole[0][1]
        z_im4 = f_mm + pinhole[0][2]
        world_coords[titles[3]].append([key, x_im4, y_im4, z_im4])

    return world_coords


In [ ]:
# RUN - define world coordinates for every image
world_coords = world_coords(titles, pixel_coords)
pprint.pprint(world_coords)

In [ ]:
# FUNCTION -  visualisation world coordinates
def visual_check_world_coords(world_coords, titles):
    """
    Visualizes world coordinates in two subplots, with left and right values separated.

    Parameters:
    - world_coords: Dictionary containing world coordinates for each image.
    - titles: List of strings, titles to be used for each image.
    """
    fig = plt.figure(figsize=(15, 10))
    subplot_index = 1
    
    for title in titles:
        coords = world_coords.get(title, [])
        ax = fig.add_subplot(2, 2, subplot_index, projection='3d')
        
        for label, x, y, z in coords:
            ax.scatter(x, y, z, color='r', marker='o')
            ax.text(x, y, z, label, color='black', fontsize=12)

        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title(f"Coordinates for {title}")
        ax.view_init(elev=90, azim=0)
        subplot_index += 1

    plt.tight_layout()
    plt.show()
    
    fig = plt.figure(figsize=(15, 10))
    subplot_index = 1
    
    for title in titles:
        coords = world_coords.get(title, [])
        ax = fig.add_subplot(2, 2, subplot_index, projection='3d')
        
        for label, x, y, z in coords:
            ax.scatter(x, y, z, color='r', marker='o')
            ax.text(x, y, z, label, color='black', fontsize=12)

        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title(f"Coordinates for {title}")
        ax.view_init(elev=0, azim=0)
        subplot_index += 1

    plt.tight_layout()
    plt.show()
    
    
    # Create a figure with two subplots
    fig, axs = plt.subplots(1, 2, figsize=(20, 10), subplot_kw={'projection': '3d'})

    # Define left and right titles
    left_titles = [titles[0], titles[2]]
    right_titles = [titles[1], titles[3]]

    # Loop over left and right titles and corresponding coordinates
    for ax, title_group in zip(axs, [left_titles, right_titles]):
        for title in title_group:
            # Extract x, y, z coordinates
            coords_3D = world_coords[title]
            x_coords = [coord[1] for coord in coords_3D]
            y_coords = [coord[2] for coord in coords_3D]
            z_coords = [coord[3] for coord in coords_3D]

            # Plot the data points
            ax.scatter(x_coords, y_coords, z_coords, marker='o', label=title)

        # Set labels and title for each subplot
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        if title_group == left_titles:
            ax.set_title('3D Plot Left')
        else:
            ax.set_title('3D Plot Right')
        ax.legend()
        ax.set_xlim([-50, 50])
        ax.set_ylim([-50, 50])

        # Set the azimuthal angle
        ax.view_init(elev=90, azim=180)

    # Show the plot
    plt.show()


In [ ]:
# RUN - world coords visualisation
visual_check_world_coords(world_coords, titles)

In [ ]:
# FUNCTION - define the coordinates of the real points
def define_D_mean_all(titles, world_coords, side):
    """
    Computes the mean 3D coordinates for matching labels from two sets of world coordinates.

    Parameters:
    - titles: List of titles for the images, in the order [left_image1, right_image1, left_image2, right_image2].
    - world_coords: Dictionary with titles as keys and lists of tuples as values. Each tuple contains
      a label and its corresponding (x, y, z) coordinates.
    - side: String indicating which side to consider ('left' or 'right').

    Returns:
    - D_mean_all: List of tuples, where each tuple contains a label and the mean (x, y, z) coordinates
      of the corresponding points from the selected titles.
    """
    if side == 'left':
        selected_titles = [titles[0], titles[2]]
    elif side == 'right':
        selected_titles = [titles[1], titles[3]]

    D_mean_all = []

    for point1 in world_coords[selected_titles[0]]:
        for point2 in world_coords[selected_titles[1]]:
            label1 = point1[0]
            label2 = point2[0]
            
            if label1 == label2:
                x1, y1, z1 = point1[1], point1[2], point1[3]
                x2, y2, z2 = point2[1], point2[2], point2[3]
                
                A = np.array([[pinhole[0][0] - x2, x1 - pinhole[1][0]],
                              [-y2, y1],
                              [-z2, z1]])
                B = np.array([[pinhole[1][0] - pinhole[0][0]],
                              [pinhole[1][1] - pinhole[0][1]],
                              [pinhole[1][2] - pinhole[0][2]]])

                t_est, residuals, rank, singular_values = np.linalg.lstsq(A, B, rcond=None)

                t2 = t_est[0]
                t1 = t_est[1]

                x_3D_2 = pinhole[0][0] + t2 * (pinhole[0][0] - x2)
                y_3D_2 = pinhole[0][1] + t2 * (pinhole[0][1] - y2)
                z_3D_2 = pinhole[0][2] + t2 * (pinhole[0][2] - z2)

                D2 = np.array([x_3D_2, y_3D_2, z_3D_2])

                x_3D_1 = pinhole[1][0] + t1 * (pinhole[1][0] - x1)
                y_3D_1 = pinhole[1][1] + t1 * (pinhole[1][1] - y1)
                z_3D_1 = pinhole[1][2] + t1 * (pinhole[1][2] - z1)

                D1 = np.array([x_3D_1, y_3D_1, z_3D_1])

                D_mean = (D1 + D2) / 2
                
                x = D_mean[0][0]
                y = D_mean[1][0]
                z = D_mean[2][0]

                # Add key (label) to D_mean_all
                D_mean_all.append((label1, [x, y, z]))

    return D_mean_all


In [ ]:
# RUN - define real 3D coordinates left & right leaf
D_mean_all_left = define_D_mean_all(titles, world_coords, 'left')
D_mean_all_right = define_D_mean_all(titles, world_coords, 'right')

print("3D coordinates left leaf:")
pprint.pprint(D_mean_all_left)

print("3D coordinates right leaf")
pprint.pprint(D_mean_all_right)

In [ ]:
# FUNCTION - visualization 3D coordinates
def visual_check_3D_coords(D_mean_all):
    """
    Visualises 3D coordinates with colour based on height.

    Parameters:
    D_mean_all (list): A list of tuples, where each tuple consists of a label and a list of [x, y, z] coordinates.
    """
    # Calculate min/max for x,y,z
    x_values = [d[1][0] for d in D_mean_all]
    y_values = [d[1][1] for d in D_mean_all]
    z_values = [d[1][2] for d in D_mean_all]

    x_min, x_max = min(x_values), max(x_values)
    y_min, y_max = min(y_values), max(y_values)
    z_min, z_max = min(z_values), max(z_values)

    fig = plt.figure(figsize=(15, 10))
    ax = fig.add_subplot(111, projection='3d')

    for key, coords in D_mean_all:
        x_3D, y_3D, z_3D = coords[0], coords[1], coords[2]
        norm = (z_3D - z_min) / (z_max - z_min)
        color = [norm, 1 - norm, 0]
        ax.scatter(x_3D, y_3D, z_3D, color=color, marker='o')
        ax.text(x_3D, y_3D, z_3D, key, color='black', fontsize=10)
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_xlim([x_min - 10, x_max + 10])
    ax.set_ylim([y_min - 10, y_max + 10])
    ax.set_zlim([z_min - 10, z_max + 10])
    ax.set_title('3D Plot')

    ax.view_init(elev=0, azim=90)

    plt.tight_layout()
    plt.show()

    azimuth_angles = [0, 45, 90, 120, 150, 180]
    fig, axes = plt.subplots(2, 3, figsize=(15, 10), subplot_kw={'projection': '3d'})

    for ax, azim in zip(axes.flatten(), azimuth_angles):
        for key, coords in D_mean_all:
            x_3D, y_3D, z_3D = coords[0], coords[1], coords[2]
            norm = (z_3D - z_min) / (z_max - z_min)
            color = [norm, 1 - norm, 0]
            ax.scatter(x_3D, y_3D, z_3D, color=color, marker='o')
            ax.text(x_3D, y_3D, z_3D, key, color='black', fontsize=10)
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_xlim([x_min - 10, x_max + 10])
        ax.set_ylim([y_min - 10, y_max + 10])
        ax.set_zlim([z_min - 10, z_max + 10])
        ax.view_init(elev=0, azim=azim)
        ax.set_title(f'3D Plot (Azimuth {azim})')

    plt.tight_layout()
    plt.show()


In [ ]:
# RUN - visualization 3D coordinates left leaf
visual_check_3D_coords(D_mean_all_left)

In [ ]:
# RUN - visualization 3D coordinates right leaf
visual_check_3D_coords(D_mean_all_right)

In [ ]:
# FUNCTION - plot regression plane through points
def plot_best_fit_plane(D_mean_all, visualize=False):
    """
    Computes the best-fit plane for a set of 3D points and optionally visualizes it.

    Parameters:
    - D_mean_all: List of tuples, where each tuple contains a label and the (x, y, z) coordinates of a point.
    - visualize: Boolean flag indicating whether to visualize the points and the best-fit plane.

    Returns:
    - C: Coefficients of the best-fit plane equation ax + by + cz + d = 0.
    """
    # Calculate the min and max values for x, y, and z coordinates
    x_values = [d[1][0] for d in D_mean_all]
    y_values = [d[1][1] for d in D_mean_all]
    z_values = [d[1][2] for d in D_mean_all]

    x_min, x_max = min(x_values), max(x_values)
    y_min, y_max = min(y_values), max(y_values)
    z_min, z_max = min(z_values), max(z_values)

    # Azimuthal angles for the subplots
    azimuth_angles = [0, 45, 90, 120, 150, 180]

    fig, axes = plt.subplots(2, 3, figsize=(15, 10), subplot_kw={'projection': '3d'})
    
    coordinates = [point[1] for point in D_mean_all]
    labels = [point[0] for point in D_mean_all]
    D_mean_array = np.array(coordinates)

    A = np.c_[D_mean_array[:, 0], D_mean_array[:, 1], np.ones(D_mean_array.shape[0])]
    C, _, _, _ = scipy.linalg.lstsq(A, D_mean_array[:, 2])

    a, b, c = C[0], C[1], -1
    d = -C[2]

    mn = np.min(D_mean_array, axis=0)
    mx = np.max(D_mean_array, axis=0)
    X, Y = np.meshgrid(np.linspace(mn[0], mx[0], 20), np.linspace(mn[1], mx[1], 20))

    Z = (d - a*X - b*Y) / c

    if visualize:        
        for i, (ax, azim) in enumerate(zip(axes.flatten(), azimuth_angles)):
            ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.2)
            ax.scatter(D_mean_array[:, 0], D_mean_array[:, 1], D_mean_array[:, 2], c='red', label='Points')
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            ax.set_zlabel('Z')
            ax.set_title(f'Best-fit Plane and Points (Azim = {azim}°)')
            ax.view_init(elev=0, azim=azim)

    plt.show()
    
    return C


In [ ]:
# RUN - plot regression plane left leaf
plot_best_fit_plane(D_mean_all_left, True)

In [ ]:
# RUN - plot regression plane right leaf
plot_best_fit_plane(D_mean_all_right, True)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visual_check_both_leaves(D_mean_all_left, D_mean_all_right):
    """
    Visualizes the 3D coordinates of points from two sets of leaves (left and right) in multiple views.

    Parameters:
    - D_mean_all_left: List of tuples, where each tuple contains a label and the (x, y, z) coordinates 
      of a point from the left leaf.
    - D_mean_all_right: List of tuples, where each tuple contains a label and the (x, y, z) coordinates 
      of a point from the right leaf.

    Returns:
    - None
    """
    fig = plt.figure(figsize=(15, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Extracting all coordinates to determine the limits
    all_coords = [coords for key, coords in D_mean_all_left + D_mean_all_right]
    x_coords, y_coords, z_coords = zip(*all_coords)

    x_min, x_max = min(x_coords) - 15, max(x_coords) + 15
    y_min, y_max = min(y_coords) - 15, max(y_coords) + 15
    z_min, z_max = min(z_coords) - 15, max(z_coords) + 15

    # Plot shifted points from D_mean_all_left
    for key, (x_3D, y_3D, z_3D) in D_mean_all_left:
        ax.scatter(x_3D, y_3D, z_3D, color='red', marker='o')
        ax.text(x_3D, y_3D, z_3D, key, color='black', fontsize=10)

    # Plot shifted points from D_mean_all_right
    for key, (x_3D, y_3D, z_3D) in D_mean_all_right:
        ax.scatter(x_3D, y_3D, z_3D, color='blue', marker='o')
        ax.text(x_3D, y_3D, z_3D, key, color='black', fontsize=10)

    elev = 90
    azim = 180
    ax.view_init(elev=elev, azim=azim)

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = x_limits[1] - x_limits[0]
    y_range = y_limits[1] - y_limits[0]
    z_range = z_limits[1] - z_limits[0]

    max_range = max(x_range, y_range, z_range)

    ax.set_xlim3d([x_limits[0], x_limits[0] + max_range])
    ax.set_ylim3d([y_limits[0], y_limits[0] + max_range])
    ax.set_zlim3d([z_limits[0], z_limits[0] + max_range])

    plt.tight_layout()

    plt.show()

    azimuth_angles = [0, 45, 90, 120, 150, 180]

    fig, axes = plt.subplots(2, 3, figsize=(15, 10), subplot_kw={'projection': '3d'})

    for ax, azim in zip(axes.flatten(), azimuth_angles):
        for key, (x_3D, y_3D, z_3D) in D_mean_all_left:
            ax.scatter(x_3D, y_3D, z_3D, color='red', marker='o')
            ax.text(x_3D, y_3D, z_3D, key, color='black', fontsize=10)
        
        for key, (x_3D, y_3D, z_3D) in D_mean_all_right:
            ax.scatter(x_3D, y_3D, z_3D, color='blue', marker='o')
            ax.text(x_3D, y_3D, z_3D, key, color='black', fontsize=10)
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title(f'3D Plot with labels (Azim = {azim} °)')
        
        # Dynamic limits
        ax.set_xlim([x_min, x_max])
        ax.set_ylim([y_min, y_max])
        ax.set_zlim([z_min, z_max])

        ax.view_init(elev=0, azim=azim+180)

    plt.tight_layout()
    plt.show()


In [ ]:
# RUN - visualization both leaves
visual_check_both_leaves(D_mean_all_left, D_mean_all_right)

In [ ]:
# FUNCTION - pointshift
def shifted(D_mean_all):
    """
    Shifts the y-coordinates of the points in D_mean_all based on whether they are from the left or right set of leaves.

    Parameters:
    - D_mean_all: List of tuples, where each tuple contains a label and the (x, y, z) coordinates 
      of a point. It can be either from the left or right set of leaves.

    Returns:
    - D_mean_all_shifted: List of tuples, where each tuple contains a label and the shifted (x, y, z) coordinates.
    """
    D_mean_all_shifted = []
    if D_mean_all == D_mean_all_left:
        for letter, coordinates in D_mean_all:
            x = coordinates[0]
            y = coordinates[1]+65
            z = coordinates[2]
            D_mean_all_shifted.append((letter, [x, y, z]))

    elif D_mean_all == D_mean_all_right:
        for letter, coordinates in D_mean_all:
            x = coordinates[0]
            y = coordinates[1]-65
            z = coordinates[2]
            D_mean_all_shifted.append((letter, [x, y, z]))
    
    return(D_mean_all_shifted)

In [ ]:
# RUN - pointshift
D_mean_all_left_shifted = shifted(D_mean_all_left)
D_mean_all_right_shifted = shifted(D_mean_all_right)

print("Shift points left leaf:")
pprint.pprint(D_mean_all_left_shifted) 
print("Shift points right leaf:")
pprint.pprint(D_mean_all_right_shifted)

In [ ]:
# RUN - plot regression plane left leaf 
plot_best_fit_plane(D_mean_all_left_shifted, True)

In [ ]:
# RUN - plot regression plane right leaf 
plot_best_fit_plane(D_mean_all_right_shifted, True)

In [ ]:
# Visualisation shifted points
visual_check_both_leaves(D_mean_all_left_shifted, D_mean_all_right_shifted)

In [ ]:
# FUNCTION - define min, max, mean of the coordinates
def find_min_max_mean(points):
    x_values = [point[1][0] for point in points]
    y_values = [point[1][1] for point in points]
    z_values = [point[1][2] for point in points]
    
    x_mean = max(x_values)-min(x_values)
    y_mean = max(y_values)-min(y_values)
    z_mean = max(z_values)-min(z_values)
    
    return {"x_min": min(x_values),
            "x_max": max(x_values),
            "x_mean": x_mean,
            "y_min": min(y_values),
            "y_max": max(y_values),
            "y_mean": y_mean,
            "z_min": min(z_values),
            "z_max": max(z_values),
            "z_mean": z_mean,}

In [ ]:
print("Coordinates left shifted leaf")
find_min_max_mean(D_mean_all_left_shifted)

In [ ]:
print("Coordinates right shifted leaf")
find_min_max_mean(D_mean_all_right_shifted)

In [ ]:
# FUNCTION - plot both regression planes
def best_fit_planes_combined(D_mean_all_left, D_mean_all_right, visualize=False):
    """
    Computes the best-fit planes for two sets of 3D points (left and right) and optionally visualizes them.

    Parameters:
    - D_mean_all_left: List of tuples, where each tuple contains a label and the (x, y, z) coordinates 
      of a point from the left set of leaves.
    - D_mean_all_right: List of tuples, where each tuple contains a label and the (x, y, z) coordinates 
      of a point from the right set of leaves.
    - visualize: Boolean flag indicating whether to visualize the points and the best-fit planes.

    Returns:
    - plane_coeffs_left: Coefficients (a, b, c, d) of the best-fit plane for the left set of points.
    - plane_coeffs_right: Coefficients (a, b, c, d) of the best-fit plane for the right set of points.
    """
    # LEFT
    coordinates_left = [point[1] for point in D_mean_all_left]
    D_mean_array_left = np.array(coordinates_left)
    A_left = np.c_[D_mean_array_left[:, 0], D_mean_array_left[:, 1], np.ones(D_mean_array_left.shape[0])]
    C_left, _, _, _ = scipy.linalg.lstsq(A_left, D_mean_array_left[:, 2])
    a_left, b_left, c_left = C_left[0], C_left[1], -1
    d_left = -C_left[2]
    
    # RIGHT
    coordinates_right = [point[1] for point in D_mean_all_right]
    D_mean_array_right = np.array(coordinates_right)
    A_right = np.c_[D_mean_array_right[:, 0], D_mean_array_right[:, 1], np.ones(D_mean_array_right.shape[0])]
    C_right, _, _, _ = scipy.linalg.lstsq(A_right, D_mean_array_right[:, 2])
    a_right, b_right, c_right = C_right[0], C_right[1], -1
    d_right = -C_right[2]
    
    plane_coeffs_left = a_left, b_left, c_left, d_left
    plane_coeffs_right = a_right, b_right, c_right, d_right
    
    print(f"Coefficients for D_mean_all_left: a = {a_left}, b = {b_left}, c = {c_left}, d = {d_left}")
    print(f"Coefficients for D_mean_all_right: a = {a_right}, b = {b_right}, c = {c_right}, d = {d_right}")
    
    print(f"Equation of the plane for D_mean_all_left: {a_left:.2f}x + {b_left:.2f}y + {c_left:.2f}z = {d_left:.2f}")
    print(f"Equation of the plane for D_mean_all_right: {a_right:.2f}x + {b_right:.2f}y + {c_right:.2f}z = {d_right:.2f}")
    
    # Azimuthal angles for the subplots
    azimuth_angles = [0, 45, 90, 120, 150, 180]
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10), subplot_kw={'projection': '3d'})
    
    # FUNCTION - plot plane
    def plot_plane(ax, D_mean_all_left, D_mean_all_right, plane_coeffs_left, plane_coeffs_right, azim):
        """
        Plots two planes and their corresponding points in 3D space.
    
        Parameters:
        ax (Axes3D): The 3D axes on which to plot.
        D_mean_all_left (list): List of tuples with labels and coordinates of points for the left plane.
        D_mean_all_right (list): List of tuples with labels and coordinates of points for the right plane.
        plane_coeffs_left (tuple): Coefficients (a, b, c, d) of the left plane equation ax + by + cz = d.
        plane_coeffs_right (tuple): Coefficients (a, b, c, d) of the right plane equation ax + by + cz = d.
        azim (float): Azimuthal angle for the 3D plot view.  
        """
        coordinates_left = [point[1] for point in D_mean_all_left]
        labels_left = [point[0] for point in D_mean_all_left]
        D_mean_array_left = np.array(coordinates_left)
        
        coordinates_right = [point[1] for point in D_mean_all_right]
        labels_right = [point[0] for point in D_mean_all_right]
        D_mean_array_right = np.array(coordinates_right)
        
        a_left, b_left, c_left, d_left = plane_coeffs_left
        a_right, b_right, c_right, d_right = plane_coeffs_right
        
        mn_left = np.min(D_mean_array_left, axis=0)
        mx_left = np.max(D_mean_array_left, axis=0)
        
        mn_right = np.min(D_mean_array_right, axis=0)
        mx_right = np.max(D_mean_array_right, axis=0)
        
        
        # Reducing the size of the planes
        X_left, Y_left = np.meshgrid(np.linspace(mn_left[0], mx_left[0], 10), np.linspace(mn_left[1], mx_left[1], 10))
        X_right, Y_right = np.meshgrid(np.linspace(mn_right[0], mx_right[0], 10), np.linspace(mn_right[1], mx_right[1], 10))
        
        Z_left = (d_left - a_left*X_left - b_left*Y_left) / c_left
        Z_right = (d_right - a_right*X_right - b_right*Y_right) / c_right
        
        ax.plot_surface(X_left, Y_left, Z_left, rstride=1, cstride=1, alpha=0.2, color='red')
        ax.plot_surface(X_right, Y_right, Z_right, rstride=1, cstride=1, alpha=0.2, color='blue')
        
        ax.scatter(D_mean_array_left[:, 0], D_mean_array_left[:, 1], D_mean_array_left[:, 2], c='r', s=50)
        ax.scatter(D_mean_array_right[:, 0], D_mean_array_right[:, 1], D_mean_array_right[:, 2], c='b', s=50)
        
        for label, (x, y, z) in zip(labels_left, coordinates_left):
            ax.text(x, y, z, label, color='black', fontsize=12, ha='center', va='bottom')
        
        for label, (x, y, z) in zip(labels_right, coordinates_right):
            ax.text(x, y, z, label, color='black', fontsize=12, ha='center', va='bottom')
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title(f'Azim = {azim}°')
        ax.set_xlim([-80, 60])
        ax.set_ylim([-80, 60])
        ax.set_zlim([-500, -400])
        ax.view_init(elev=0, azim=azim)
    
    if visualize:     
        for ax, azim in zip(axes.flatten(), azimuth_angles):
            plot_plane(ax, D_mean_all_left, D_mean_all_right, plane_coeffs_left, plane_coeffs_right, azim)   
        plt.tight_layout()
        plt.show()
        
    return plane_coeffs_left, plane_coeffs_right



In [ ]:
# RUN - define/plot regression plane for both leaves 
plane_coeffs_left, plane_coeffs_right = best_fit_planes_combined(D_mean_all_left, D_mean_all_right, True)

In [ ]:
# RUN - define/plot regression plane for both shifted leaves
plane_coeffs_left_shifted, plane_coeffs_right_shifted = best_fit_planes_combined(D_mean_all_left_shifted, D_mean_all_right_shifted, True)

In [ ]:
# FUNCTION - correction of the spectral index
def correction_spectral_index(si, si_left, si_right, A_left, A_right, plane_coeffs_left, plane_coeffs_right):
    """
    Corrects the spectral index values for two sets of leaves (left and right) based on the orientation of their best-fit planes.

    Parameters:
    - si: Original spectral index value.
    - si_left: Spectral index value for the left set of leaves.
    - si_right: Spectral index value for the right set of leaves.
    - A_left: Area or another relevant parameter for the left set of leaves.
    - A_right: Area or another relevant parameter  for the right set of leaves.
    - plane_coeffs_left: Coefficients (a, b, c, d) of the best-fit plane for the left set of points.
    - plane_coeffs_right: Coefficients (a, b, c, d) of the best-fit plane for the right set of points.

    Returns:
    - si: Original spectral index value.
    - si_correct: Corrected spectral index value.
    """
    a_left, b_left, c_left, _ = plane_coeffs_left  
    cos_theta_left = c_left/np.sqrt(a_left**2 + b_left**2 + c_left**2)
    theta_left_radians = np.arccos(cos_theta_left)
    theta_left_degrees = math.degrees(theta_left_radians)
    correction_factor_left = 1/abs(cos_theta_left)
    
    a_right, b_right, c_right, _ = plane_coeffs_right
    cos_theta_right = c_right/np.sqrt(a_right**2 + b_right**2 + c_right**2)
    theta_right_radians = np.arccos(cos_theta_right)
    theta_right_degrees = math.degrees(theta_right_radians)
    correction_factor_right = 1/abs(cos_theta_right)
    
    print("Angle left leaf (°):", theta_left_degrees)
    print("Angle right leaf (°):", theta_right_degrees)
    print("Cosine angle left:", cos_theta_left)
    print("Cosine angle right:", cos_theta_right)
    print("Correction factor left leaf:", correction_factor_left)
    print("Correction factor right leaf:", correction_factor_right)

    A_corr_left = A_left * correction_factor_left
    A_corr_right = A_right * correction_factor_right
    A_corr_total = A_corr_left + A_corr_right
    print("Corrected Area Left:", A_corr_left)
    print("Corrected Area Right:", A_corr_right)
    print("Corrected Area Total:", A_corr_total)
    
    # Corrected value
    si_correct = (A_corr_left/A_corr_total*si_left + A_corr_right/A_corr_total*si_right)
    
    return si, si_correct


In [ ]:
# INFORMATION SPECTRAL INDICES
A_left = 170581
A_right = 120674

# FvFm
si = 'FvFm'
si_left = 0.708
si_right = 0.71

si_FvFm, si_correct_FvFm = correction_spectral_index(si, si_left, si_right, A_left, A_right, plane_coeffs_left, plane_coeffs_right) 

print(si_FvFm, si_correct_FvFm)
print()

# ChlIdx
si = 'ChlIdx'
si_left = 1.056
si_right = 1.073

si_ChlIdx, si_correct_ChlIdx = correction_spectral_index(si, si_left, si_right, A_left, A_right, plane_coeffs_left, plane_coeffs_right) 

print(si_ChlIdx, si_correct_ChlIdx)
print()

# Chl
si = 'Chl'
si_left = 2846
si_right = 2393

si_Chl, si_correct_Chl = correction_spectral_index(si, si_left, si_right, A_left, A_right, plane_coeffs_left, plane_coeffs_right) 

print(si_Chl, si_correct_Chl)
print()

## COMPARISON WITH AGISOFT

In [ ]:
def set_axes_equal(ax):
    """
    Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    """

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

In [ ]:
filePoints = f'{folderDATA}/DATA_THESIS_MARTEN/Agisoft/dense.obj'

f = open(filePoints, "r")
points = np.array([[float(x) for x in  line.strip().split(' ')[1:]]
          for line in f.readlines()[::2]])

# Plot points
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points[:,0], points[:,1], points[:,2], marker='.', s=2)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=0, azim=-90)
set_axes_equal(ax)
plt.tight_layout()
plt.show()

In [ ]:
# Split ground from leaves
treshold = -6
points_leaves = np.array([point for point in points if point[2] > treshold])
points_ground = np.array([point for point in points if point[2] <= treshold])

In [ ]:
# Plot separated ground and leaves
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points_leaves[:,0], points_leaves[:,1], points_leaves[:,2], marker='.', s=2, color='green')
ax.scatter(points_ground[:,0], points_ground[:,1], points_ground[:,2], marker='.', s=2, color='yellow')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=0, azim=-90)
plt.tight_layout()
plt.show()

In [ ]:
# Plot leaves
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points_leaves[:,0], points_leaves[:,1], points_leaves[:,2], marker='.', s=2, color='green')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=0, azim=-90)
plt.tight_layout()
plt.show()

In [ ]:
# Split two leaves
treshold = 1.35
points_right = np.array([point for point in points_leaves if point[0] > treshold])
points_left = np.array([point for point in points_leaves if point[0] <= treshold])

# Plot separated leaves
# right = yellow
# left = blue
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points_right[:,0], points_right[:,1], points_right[:,2], marker='.', s=2, color='blue')
ax.scatter(points_left[:,0], points_left[:,1], points_left[:,2], marker='.', s=2, color='red')
#ax.scatter(points_ground[:,0], points_ground[:,1], points_ground[:,2], marker='.', s=2, color='red')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=90, azim=-90)

plt.tight_layout()
plt.show()


In [ ]:
# Plot separated leaves & ground
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points_right[:,0], points_right[:,1], points_right[:,2], marker='.', s=2, color='blue')
ax.scatter(points_left[:,0], points_left[:,1], points_left[:,2], marker='.', s=2, color='red')
ax.scatter(points_ground[:,0], points_ground[:,1], points_ground[:,2], marker='.', s=2, color='yellow')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.view_init(elev=0, azim=-90)

plt.tight_layout()
plt.show()

In [ ]:
# 3D coordinates for the single leaves and ground
D_mean_all_points_right = [(f'point_{i}', (points_right[i, 0], points_right[i, 1], points_right[i, 2])) for i in range(points_right.shape[0])]
D_mean_all_points_left = [(f'point_{i}', (points_left[i, 0], points_left[i, 1], points_left[i, 2])) for i in range(points_left.shape[0])]
D_mean_all_ground = [(f'point_{i}', (points_ground[i, 0], points_ground[i, 1], points_ground[i, 2])) for i in range(points_ground.shape[0])]

coefficients_right = plot_best_fit_plane(D_mean_all_points_right, visualize=True)
print("Coefficients of the best-fit plane:", coefficients_right)

coefficients_left = plot_best_fit_plane(D_mean_all_points_left, visualize=True)
print("Coefficients of the best-fit plane:", coefficients_left)

coefficients_ground = plot_best_fit_plane(D_mean_all_ground, visualize=True)
print("Coefficients of the best-fit plane:", coefficients_ground)

In [ ]:
# FUNCTION - define regression planes agisoft
def best_fit_planes_combined_agisoft(D_mean_all_right, D_mean_all_left, visualize=False):
    """
    Computes the best-fit planes for left sets of 3D points (right and left) and optionally visualizes them.

    Parameters:
    - D_mean_all_right: List of tuples, where each tuple contains a label and the (x, y, z) coordinates 
      of a point from right set of leaves.
    - D_mean_all_left: List of tuples, where each tuple contains a label and the (x, y, z) coordinates 
      of a point from the other set of leaves.
    - visualize: Boolean flag indicating whether to visualize the points and the best-fit planes.

    Returns:
    - plane_coeffs_right: Coefficients (a, b, c, d) of the best-fit plane for the right set of points.
    - plane_coeffs_left: Coefficients (a, b, c, d) of the best-fit plane for the other set of points.
    """
    # Right
    coordinates_right = [point[1] for point in D_mean_all_right]
    D_mean_array_right = np.array(coordinates_right)
    A_right = np.c_[D_mean_array_right[:, 0], D_mean_array_right[:, 1], np.ones(D_mean_array_right.shape[0])]
    C_right, _, _, _ = scipy.linalg.lstsq(A_right, D_mean_array_right[:, 2])
    a_right, b_right, c_right = C_right[0], C_right[1], -1
    d_right = -C_right[2]
    
    # Left
    coordinates_left = [point[1] for point in D_mean_all_left]
    D_mean_array_left = np.array(coordinates_left)
    A_left = np.c_[D_mean_array_left[:, 0], D_mean_array_left[:, 1], np.ones(D_mean_array_left.shape[0])]
    C_left, _, _, _ = scipy.linalg.lstsq(A_left, D_mean_array_left[:, 2])
    a_left, b_left, c_left = C_left[0], C_left[1], -1
    d_left = -C_left[2]
    
    plane_coeffs_right = a_right, b_right, c_right, d_right
    plane_coeffs_left = a_left, b_left, c_left, d_left
    
    print(f"Coefficients for D_mean_all_right: a = {a_right}, b = {b_right}, c = {c_right}, d = {d_right}")
    print(f"Coefficients for D_mean_all_left: a = {a_left}, b = {b_left}, c = {c_left}, d = {d_left}")
    
    print(f"Equation of the plane for D_mean_all_right: {a_right:.2f}x + {b_right:.2f}y + {c_right:.2f}z = {d_right:.2f}")
    print(f"Equation of the plane for D_mean_all_left: {a_left:.2f}x + {b_left:.2f}y + {c_left:.2f}z = {d_left:.2f}")
    
    # Azimuthal angles for the subplots
    azimuth_angles = [0, 45, 90, 120, 150, 180]
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10), subplot_kw={'projection': '3d'})
    
    def plot_plane(ax, D_mean_all_right, D_mean_all_left, plane_coeffs_right, plane_coeffs_left, azim):
        """
        Plots left planes and their corresponding points in 3D space.

        Parameters:
        ax (Axes3D): The 3D axes on which to plot.
        D_mean_all_right (list): List of tuples with labels and coordinates of points for the first plane.
                               Example: [(label1, [x1, y1, z1]), (label2, [x2, y2, z2]), ...]
        D_mean_all_left (list): List of tuples with labels and coordinates of points for the second plane.
                               Example: [(label1, [x1, y1, z1]), (label2, [x2, y2, z2]), ...]
        plane_coeffs_right (tuple): Coefficients (a, b, c, d) of the first plane equation ax + by + cz = d.
        plane_coeffs_left (tuple): Coefficients (a, b, c, d) of the second plane equation ax + by + cz = d.
        azim (float): Azimuthal angle for the 3D plot view.
        """
        coordinates_right = [point[1] for point in D_mean_all_right]
        labels_right = [point[0] for point in D_mean_all_right]
        D_mean_array_right = np.array(coordinates_right)
        
        coordinates_left = [point[1] for point in D_mean_all_left]
        labels_left = [point[0] for point in D_mean_all_left]
        D_mean_array_left = np.array(coordinates_left)
        
        a_right, b_right, c_right, d_right = plane_coeffs_right
        a_left, b_left, c_left, d_left = plane_coeffs_left
        
        mn_right = np.min(D_mean_array_right, axis=0)
        mx_right = np.max(D_mean_array_right, axis=0)
        
        mn_left = np.min(D_mean_array_left, axis=0)
        mx_left = np.max(D_mean_array_left, axis=0)
        
        # Reducing the size of the planes
        X_right, Y_right = np.meshgrid(np.linspace(mn_right[0], mx_right[0], 10), np.linspace(mn_right[1], mx_right[1], 10))
        X_left, Y_left = np.meshgrid(np.linspace(mn_left[0], mx_left[0], 10), np.linspace(mn_left[1], mx_left[1], 10))
        
        Z_right = (d_right - a_right*X_right - b_right*Y_right) / c_right
        Z_left = (d_left - a_left*X_left - b_left*Y_left) / c_left
        
        ax.plot_surface(X_right, Y_right, Z_right, rstride=1, cstride=1, alpha=0.2, color='blue')
        ax.plot_surface(X_left, Y_left, Z_left, rstride=1, cstride=1, alpha=0.2, color='red')
        
        ax.scatter(D_mean_array_right[:, 0], D_mean_array_right[:, 1], D_mean_array_right[:, 2], c='b', s=50)
        ax.scatter(D_mean_array_left[:, 0], D_mean_array_left[:, 1], D_mean_array_left[:, 2], c='r', s=50)
        
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_title(f'Azim = {azim}°')
        ax.view_init(elev=0, azim=azim+180)
    
    if visualize:     
        for ax, azim in zip(axes.flatten(), azimuth_angles):
            plot_plane(ax, D_mean_all_right, D_mean_all_left, plane_coeffs_right, plane_coeffs_left, azim)   
        plt.tight_layout()
        plt.show()
        
    return plane_coeffs_right, plane_coeffs_left



In [ ]:
# RUN - plot regression plane for both leaves
plane_coeffs_right, plane_coeffs_left = best_fit_planes_combined_agisoft(D_mean_all_points_right, D_mean_all_points_left, True)

In [ ]:
def angle_with_groundplane(D_mean_all_points_leaf, D_mean_all_points_ground):
    """
    Calculates the coefficients of the planes formed by leaf points and ground points,
    and the angle between these two planes.

    Inputs:
    D_mean_all_points_leaf: List of tuples, where each tuple contains an identifier and a list of 3D coordinates representing points on the leaf.
        Example: [(id1, [x1, y1, z1]), (id2, [x2, y2, z2]), ...]
    D_mean_all_points_ground: List of tuples, where each tuple contains an identifier and a list of 3D coordinates representing points on the ground.
        Example: [(id1, [x1, y1, z1]), (id2, [x2, y2, z2]), ...]

    Outputs:
    plane_coeffs_leaf: List of coefficients [a_leaf, b_leaf, c_leaf, d_leaf] for the plane equation of the leaf.
    plane_coeffs_ground: List of coefficients [a_ground, b_ground, c_ground, d_ground] for the plane equation of the ground.
    theta_leaf_degrees: The angle in degrees between the leaf plane and the ground plane.
    """
    # LEAF
    coordinates_leaf = [point[1] for point in D_mean_all_points_leaf]
    D_mean_array_leaf = np.array(coordinates_leaf)
    A_leaf = np.c_[D_mean_array_leaf[:, 0], D_mean_array_leaf[:, 1], np.ones(D_mean_array_leaf.shape[0])]
    C_leaf, _, _, _ = scipy.linalg.lstsq(A_leaf, D_mean_array_leaf[:, 2])
    a_leaf, b_leaf, c_leaf = C_leaf[0], C_leaf[1], -1
    d_leaf = -C_leaf[2]

    # GROUND
    coordinates_ground = [point[1] for point in D_mean_all_points_ground]
    D_mean_array_ground = np.array(coordinates_ground)
    A_ground = np.c_[D_mean_array_ground[:, 0], D_mean_array_ground[:, 1], np.ones(D_mean_array_ground.shape[0])]
    C_ground, _, _, _ = scipy.linalg.lstsq(A_ground, D_mean_array_ground[:, 2])
    a_ground, b_ground, c_ground = C_ground[0], C_ground[1], -1
    d_ground = -C_ground[2]

    plane_coeffs_leaf = [a_leaf, b_leaf, c_leaf, d_leaf]
    plane_coeffs_ground = [a_ground, b_ground, c_ground, d_ground]
    
    u = [a_leaf, b_leaf, c_leaf]
    v = [a_ground, b_ground, c_ground]
    print(u)
    print(v)
    dot_product = np.dot(u,v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    cos_theta_leaf = dot_product/(norm_u*norm_u)
    theta_leaf_radians = np.arccos(cos_theta_leaf)
    theta_leaf_degrees = math.degrees(theta_leaf_radians)
    
    return plane_coeffs_leaf, plane_coeffs_ground, theta_leaf_radians, theta_leaf_degrees

In [ ]:
# RUN - coefficients of the plane and calculation of the angle
plane_coeffs_leaf_right, plane_coeffs_ground, agi_theta_leaf_right_radians, agi_theta_leaf_right_degrees = angle_with_groundplane(D_mean_all_points_right, D_mean_all_ground)

print(f"Coefficients for D_mean_all_leaf right: a = {plane_coeffs_leaf_right[0]}, b = {plane_coeffs_leaf_right[1]}, c = {plane_coeffs_leaf_right[2]}, d = {plane_coeffs_leaf_right[3]}")
print(f"Coefficients for D_mean_all_ground: a = {plane_coeffs_ground[0]}, b = {plane_coeffs_ground[1]}, c = {plane_coeffs_ground[2]}, d = {plane_coeffs_ground[3]}")
print(f"Equation of the plane for D_mean_all_leaf right: {plane_coeffs_leaf_right[0]:.2f}x + {plane_coeffs_leaf_right[1]:.2f}y + {plane_coeffs_leaf_right[2]:.2f}z = {plane_coeffs_leaf_right[3]:.2f}")
print(f"Equation of the plane for D_mean_all_ground: {plane_coeffs_ground[0]:.2f}x + {plane_coeffs_ground[1]:.2f}y + {plane_coeffs_ground[2]:.2f}z = {plane_coeffs_ground[3]:.2f}")
print("Theta leaf right (radians):", agi_theta_leaf_right_radians)
print("Theta leaf right (°):", agi_theta_leaf_right_degrees)
print()

plane_coeffs_leaf_left, plane_coeffs_ground, agi_theta_leaf_left_radians, agi_theta_leaf_left_degrees = angle_with_groundplane(D_mean_all_points_left, D_mean_all_ground)

print(f"Coefficients for D_mean_all_leaf left: a = {plane_coeffs_leaf_left[0]}, b = {plane_coeffs_leaf_left[1]}, c = {plane_coeffs_leaf_left[2]}, d = {plane_coeffs_leaf_left[3]}")
print(f"Coefficients for D_mean_all_ground: a = {plane_coeffs_ground[0]}, b = {plane_coeffs_ground[1]}, c = {plane_coeffs_ground[2]}, d = {plane_coeffs_ground[3]}")
print(f"Equation of the plane for D_mean_all_leaf left: {plane_coeffs_leaf_left[0]:.2f}x + {plane_coeffs_leaf_left[1]:.2f}y + {plane_coeffs_leaf_left[2]:.2f}z = {plane_coeffs_leaf_left[3]:.2f}")
print(f"Equation of the plane for D_mean_all_ground: {plane_coeffs_ground[0]:.2f}x + {plane_coeffs_ground[1]:.2f}y + {plane_coeffs_ground[2]:.2f}z = {plane_coeffs_ground[3]:.2f}")
print("Theta leaf left (radians):", agi_theta_leaf_left_radians)
print("Theta leaf left (°):",agi_theta_leaf_left_degrees)
print()

In [ ]:
def agi_correction_spectral_index(si, si_right, si_left, A_right, A_left, theta_leaf_right, theta_leaf_left):
    """
    Applies angle correction to the spectral index (SI) measurements.

    Inputs:
    si: The original spectral index value before correction.
    si_left: The spectral index value measured from the left view.
    si_right: The spectral index value measured from the right view.
    A_left: The amplitude (or some other weighting factor) for the left view.
    A_right: The amplitude (or some other weighting factor) for the right view.
    theta_leaf_left: The angle between the leaf plane and the ground plane for the left view (in radians).
    theta_leaf_right: The angle between the leaf plane and the ground plane for the right view (in radians).

    Outputs:
    si: The original spectral index value before correction.
    si_correct: The corrected spectral index value after applying angle correction.
    """
    
    # Given angles
    print("Angle Blue (°):", theta_leaf_right)
    print("Angle Red (°):", theta_leaf_left)
    
    # Correction factor
    cos_theta_leaf_right = np.cos(agi_theta_leaf_right_radians)
    cos_theta_leaf_left = np.cos(agi_theta_leaf_left_radians)
    correction_factor_right = 1/abs(cos_theta_leaf_right)
    correction_factor_left = 1/abs(cos_theta_leaf_left)
    print("Cosine Angle Blue:", cos_theta_leaf_right)
    print("Cosine Angle Red:", cos_theta_leaf_left)
    print("Correction factor Blue:", correction_factor_right)
    print("Correction factor Red:", correction_factor_left)

    A_corr_right = A_right * correction_factor_right
    A_corr_left = A_left * correction_factor_left
    A_corr_total = A_corr_right + A_corr_left
    print("Corrected Area Blue:", A_corr_right)
    print("Corrected Area Red:", A_corr_left)
    print("Corrected Area Total:", A_corr_total)

    # Corrected value
    si_correct = (A_corr_right/A_corr_total*si_right + A_corr_left/A_corr_total*si_left)
    
    return si, si_correct


In [ ]:
# INFORMATION - correction spectral indices
A_left = 170581
A_right = 120674

# FvFm
si = 'FvFm'
si_left = 0.708
si_right = 0.71


si_FvFm, si_correct_FvFm = agi_correction_spectral_index(si, si_right, si_left, A_right, A_left, agi_theta_leaf_right_degrees, agi_theta_leaf_left_degrees) 

print(si_FvFm, si_correct_FvFm)
print()

# ChlIdx
si = 'ChlIdx'
si_left = 1.056
si_right = 1.073

si_ChlIdx, si_correct_ChlIdx = agi_correction_spectral_index(si, si_right, si_left, A_right, A_left, agi_theta_leaf_right_degrees, agi_theta_leaf_left_degrees) 

print(si_ChlIdx, si_correct_ChlIdx)
print()

# Chl
si = 'Chl'
si_left = 2846
si_right = 2393

si_Chl, si_correct_Chl = agi_correction_spectral_index(si, si_right, si_left, A_right, A_left, agi_theta_leaf_right_degrees, agi_theta_leaf_left_degrees) 

print(si_Chl, si_correct_Chl)
print()

# CALIBRATION AGISOFT

In [ ]:
import xml.etree.ElementTree as ET
path_calib_agisoft = f'{folderDATA}/DATA_THESIS_MARTEN/Agisoft/camera_cal.xml'

with open(path_calib_agisoft, 'r', encoding='utf-8') as calib_agisoft:
    xml_data = calib_agisoft.read()
print(xml_data)

root = ET.fromstring(xml_data)

f_element = root.find('f')
f_agisoft = float(f_element.text)

cx = root.find('cx')
cy = root.find('cy')
xp_agisoft = float(cx.text)
yp_agisoft = float(cy.text)

k1 = root.find('k1')
k1_agisoft = float(k1.text)
k2 = root.find('k2')
k2_agisoft = float(k2.text)
k3 = root.find('k3')
k3_agisoft = float(k3.text)
p1 = root.find('p1')
p1_agisoft = float(p1.text)
p2 = root.find('p2')
p2_agisoft = float(p2.text)


In [ ]:
# OpenCv calibration
print("OpenCv:", f_pix, xp_pix, yp_pix)

# Agisoft calibration (pixels?)
print("Agisoft:", f_agisoft, xp_agisoft, yp_agisoft)

# difference due to 4 images for calibration

# DISTORTION MATRIX
dist_agisoft = [k1_agisoft, k2_agisoft, p1_agisoft, p2_agisoft, k3_agisoft]
print(dist_agisoft)